# Proyecto final Técnicas de inteligencia Artificial.
Integrantes:
- Leonardo Fabio Mercado Benítez.
- María Alejandra Arias Torres.

In [1]:
import cv2
import numpy as np
print('Librerias importadas correctamente')

Librerias importadas correctamente


Intento 1

In [13]:
video = cv2.VideoCapture(0)

i = 0
kernel = np.ones((5, 5), np.uint8)

while True:
    ret, frame = video.read()
    if ret == False:
        break
    cv2.imshow("Color", frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if i == 20:
        bgGray = gray
    elif i > 20:
        dif = cv2.absdiff(gray, bgGray)
        cv2.imshow('Diferencia', dif)
        _, th = cv2.threshold(dif, 40, 255, cv2.THRESH_BINARY)
        cv2.imshow("Umbralizado", th)
        opening = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
        cv2.imshow("pre-Filtrado", closing)
        
    i = i + 1
        

    if cv2.waitKey(30) & 0xFF == ord("q"):
        break
        
video.release()
cv2.destroyAllWindows();

In [4]:
video.release()
cv2.destroyAllWindows();